# SITCOM-318: Stuttered Image verification on-sky

This notebook is designed to execute the Stuttered Image verification test.

It does so by sending Scripts to the ScriptQueue that will perform the necessary operations.

In [ ]:
import yaml
import logging
import jsonschema

from lsst.ts import salobj
from lsst.ts.idl.enums.ScriptQueue import Location, SalIndex

In [ ]:
domain = salobj.Domain()

In [ ]:
sq_remote = salobj.Remote(
    domain, 
    "ScriptQueue", 
    index=SalIndex.AUX_TEL
)

In [ ]:
await sq_remote.start_task

## Scripts Configuration



In [ ]:
script_description="Executing SITCOM-318"

In [ ]:
track_target_is_standard = True
track_target_script_path = "auxtel/track_target.py"
track_target_script_config = dict(
    find_target=dict(
        az=180.0,
        el=70.0,
        mag_limit=8.0,
    ),
    offset=dict(
        x=-10.0,
        y=10.0,
    )
)

In [ ]:
take_stuttered_is_standard = True
take_stuttered_script_path = "auxtel/take_stuttered_latiss.py"
take_stuttered_script_config = dict(
    n_images=10,
    exp_time=2,
    program="SITCOM-318",
    reason="Stuttered_Imaging",
)

## Validate configuration



### Track target

In [ ]:
is_standard = track_target_is_standard
script_path = track_target_script_path
script_config = track_target_script_config

In [ ]:
sq_remote.evt_configSchema.flush()
await sq_remote.cmd_showSchema.set_start(
    isStandard=is_standard,
    path=script_path,
)
script_schema = await sq_remote.evt_configSchema.next(flush=False, timeout=5)

In [ ]:
config_validator = salobj.DefaultingValidator(schema=yaml.safe_load(script_schema.configSchema))

In [ ]:
validation_err = None

try:
    config_validator.validate(script_config)
except jsonschema.ValidationError as e:
    validation_err = e.message

if validation_err is not None:
    raise RuntimeError(validation_err)
else:
    print("Configuration OK!")

### Take Stuttered

In [ ]:
is_standard = take_stuttered_is_standard
script_path = take_stuttered_script_path
script_config = take_stuttered_script_config

In [ ]:
sq_remote.evt_configSchema.flush()
await sq_remote.cmd_showSchema.set_start(
    isStandard=is_standard,
    path=script_path,
)
script_schema = await sq_remote.evt_configSchema.next(flush=False, timeout=5)

In [ ]:
config_validator = salobj.DefaultingValidator(schema=yaml.safe_load(script_schema.configSchema))

In [ ]:
validation_err = None

try:
    config_validator.validate(script_config)
except jsonschema.ValidationError as e:
    validation_err = e.message

if validation_err is not None:
    raise RuntimeError(validation_err)
else:
    print("Configuration OK!")

## Adding Script to ScriptQueue

The next cell adds the Script to the ScriptQueue and returns as soon as the script is queued.

In [ ]:
is_standard = track_target_is_standard
script_path = track_target_script_path
script_config = track_target_script_config

In [ ]:
ack = await sq_remote.cmd_add.set_start(
    isStandard=is_standard,
    path=script_path,
    descr=script_description,
    config=yaml.safe_dump(script_config),
    location=Location.LAST,
    logLevel=logging.DEBUG,
)

In [ ]:
is_standard = take_stuttered_is_standard
script_path = take_stuttered_script_path
script_config = take_stuttered_script_config

In [ ]:
ack = await sq_remote.cmd_add.set_start(
    isStandard=is_standard,
    path=script_path,
    descr=script_description,
    config=yaml.safe_dump(script_config),
    location=Location.LAST,
    logLevel=logging.DEBUG,
)